This script shows how the trained model acts as a “digital twin.” It will consume new data points (or user-provided inputs) and predict chamber temperature in real-time.

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from IPython.display import display
from ipywidgets import interact, FloatSlider, Button, HBox, VBox
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.losses import MeanSquaredError



### Load Model and Scaler

In [40]:
model = keras.models.load_model(
    'cryogenic_temp_predictor.h5',
    custom_objects={'mse': MeanSquaredError()}
)
df_scaled_ref = pd.read_csv('cryogenic_chamber_data_scaled.csv')
feature_cols = ['chamber_temp_K', 'chamber_pressure_bar', 'external_temp_K', 'power_input']


In [41]:
# so that predicted values are scaled back to their original scale for interpretation.
scaler = MinMaxScaler()
scaler.fit(df_scaled_ref[feature_cols])

MinMaxScaler()

In [42]:
TIME_STEPS = 10

In [43]:
# We need a small buffer of the last TIME_STEPS data points to predict the next one
# For demonstration, let's take the final portion of the dataset as our "live" buffer.
df_original = pd.read_csv('cryogenic_chamber_data.csv', parse_dates=['timestamp'])
df_original_sorted = df_original.sort_values('timestamp')
live_buffer = df_original_sorted.iloc[-TIME_STEPS:].copy().reset_index(drop=True)


### Function to predict next steps

In [44]:
def digital_twin_predict_next(buffer_df):
    """Given a buffer of the last TIME_STEPS rows (unscaled), predict next chamber temperature."""
    # 1) Scale the buffer
    buffer_scaled = scaler.transform(buffer_df[feature_cols])
    
    # 2) Reshape to (1, TIME_STEPS, n_features)
    X_input = np.expand_dims(buffer_scaled, axis=0)
    
    # 3) Predict scaled temperature
    pred_scaled = model.predict(X_input)[0, 0]
    
    # 4) Inverse scale to get actual temperature
    # We'll create a placeholder for the inverse transform
    placeholder = np.zeros((1, len(feature_cols)))
    temp_idx = feature_cols.index('chamber_temp_K')
    placeholder[0, temp_idx] = pred_scaled
    pred_inv = scaler.inverse_transform(placeholder)[0, temp_idx]
    
    return pred_inv

### Live Submission Demonstration

In [45]:
next_temp_prediction = digital_twin_predict_next(live_buffer)
print("Digital Twin - Next predicted chamber temperature (K):", next_temp_prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Digital Twin - Next predicted chamber temperature (K): -0.26661896705627447


### Interactive Cell

####  Let's allow the user to tweak the next input (pressure, external temp, power) to see how the digital twin responds


In [46]:
def interactive_twin(chamber_temp, chamber_pressure, external_temp, power_input):
    # Copy the live buffer
    buffer_temp = live_buffer.copy()

    # Create a new row with user inputs
    new_row = {
        'chamber_temp_K': chamber_temp,
        'chamber_pressure_bar': chamber_pressure,
        'external_temp_K': external_temp,
        'power_input': power_input
    }

    # Shift the buffer and add the new row
    buffer_temp = pd.concat(
        [buffer_temp.iloc[1:], pd.DataFrame([new_row])],
        ignore_index=True
    )

    # Predict the next chamber temperature
    pred = digital_twin_predict_next(buffer_temp)
    print(f"Given the new inputs -> Next predicted chamber temp: {pred:.4f} K")

In [47]:
chamber_temp_slider = FloatSlider(value=10.0, min=2.0, max=30.0, step=0.5, description='Chamb. Temp (K)')
chamber_pressure_slider = FloatSlider(value=1e-5, min=1e-6, max=1e-4, step=1e-6, description='Pressure (bar)')
external_temp_slider = FloatSlider(value=300.0, min=290.0, max=310.0, step=0.5, description='Ext. Temp (K)')
power_slider = FloatSlider(value=100.0, min=0.0, max=200.0, step=5.0, description='Power')


In [50]:
interact(
    interactive_twin,
    chamber_temp=chamber_temp_slider,
    chamber_pressure=chamber_pressure_slider,
    external_temp=external_temp_slider,
    power_input=power_slider
);

interactive(children=(FloatSlider(value=10.0, description='Chamb. Temp (K)', max=30.0, min=2.0, step=0.5), Flo…